# Análisis archivo de cargas SAP

In [1]:
import pandas as pd
import datetime
import os
from tkinter import filedialog

username = os.getlogin()

hoy = datetime.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
date_formatcartola = "%d/%m/%Y"
date_formatcarga = "%Y%m%d"
fecha_actual = hoy.strftime(date_format)
fecha_cartola = hoy.strftime(date_formatexport)

folder_path = 'C:\\Users\\' + username + '\\Documents\\Py\\CARGASSAP\\ANALISIS'

print("Cargando archivos...")
df_list = []
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file),
                        sep=';',header=None,
                        engine='python',
                        #dtype='int64'
                        )
    df_list.append(df)
df_cargas = pd.concat(df_list)
df_cargas = df_cargas.dropna(axis=1)
df_cargas['agencia'] = df_cargas['agencia'].replace('AG. ','',regex=True).str.slice(0,-3)
df_cargas['agencia'] = df_cargas['agencia'].str.slice(0,-3)
df_cargas['post'] = pd.to_datetime((df_cargas['post'].astype(str)),format=date_formatcarga)
df_cargas['doc'] = pd.to_datetime((df_cargas['doc'].astype(str)),format=date_formatcarga)
                       
df_cargas.columns = ['flowid','counter','account','period','post','constant1','sa','gl','monto','indcont','user','ref','agencia','doc']

df_parc = pd.read_csv('C:\\Users\\' + username + '\\Documents\\Py\\CARGASSAP\\PARCIALIZADOS\\conteoparcializados.csv',
                        sep=',',
                        names=['agencia','count'],
                        header='infer',
                        date_format=date_formatcarga)




Cargando archivos...


In [2]:
df_adosc = df_cargas[df_cargas['monto'] == 200000]
count_adosc = df_adosc.groupby('agencia').size().reset_index(name='count').sort_values(by='count',ascending=False)
df_acien = df_cargas[df_cargas['monto'] == 100000]
count_acien = df_acien.groupby('agencia').size().reset_index(name='count').sort_values(by='count',ascending=False)


df_depositos = pd.merge(count_adosc,count_acien,on='agencia',how='outer')
df_depositos.columns = ['agencia','depdosc','depcien']
df_depositos = pd.merge(df_depositos,df_parc,on='agencia',how='left')
df_depositos['depdosc'] = df_depositos['depdosc'].fillna('0').astype(int)
df_depositos['depcien'] = df_depositos['depcien'].fillna('0').astype(int)
df_depositos['count'] = df_depositos['count'].fillna('0').astype(int)

In [3]:
df_adosc= df_adosc[df_adosc['doc'] <= '2024-12-30']
df_adosc= df_adosc[df_adosc['doc'] >= '2024-12-01']
df_adosc['monto'].sum()
freq_adosc = df_adosc.groupby(['doc', 'monto']).size().reset_index(name='frequency')
print(freq_adosc)


          doc   monto  frequency
0  2024-12-02  200000         86
1  2024-12-03  200000         89
2  2024-12-04  200000         49
3  2024-12-05  200000         67
4  2024-12-06  200000         60
5  2024-12-09  200000         76
6  2024-12-10  200000         47
7  2024-12-11  200000         28
8  2024-12-12  200000         24
9  2024-12-13  200000         30
10 2024-12-16  200000         34
11 2024-12-17  200000          6
12 2024-12-18  200000         34
13 2024-12-19  200000         34
14 2024-12-20  200000         35
15 2024-12-23  200000         34
16 2024-12-24  200000         32
17 2024-12-26  200000         23
18 2024-12-27  200000         34
19 2024-12-30  200000         59
